In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
itv_aed = pd.read_csv(
    '/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/interventions_aed_related_with_province.csv',
    low_memory=False)
aed = pd.read_csv('/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/aed_location_latlon.csv')
hospital = pd.read_csv('/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/hospital.csv')

In [4]:
itv_aed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105841 entries, 0 to 105840
Data columns (total 56 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   mission_id                        105841 non-null  int64  
 1   service_name                      93471 non-null   object 
 2   postalcode_permanence             65151 non-null   float64
 3   cityname_permanence               69218 non-null   object 
 4   streetname_permanence             69623 non-null   object 
 5   housenumber_permanence            2667 non-null    float64
 6   latitude_permanence               97007 non-null   float64
 7   longitude_permanence              97690 non-null   float64
 8   permanence_short_name             105670 non-null  object 
 9   permanence_long_name              93486 non-null   object 
 10  vector_type                       104009 non-null  object 
 11  eventtype_firstcall               65569 non-null   o

In [6]:
aed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15775 entries, 0 to 15774
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            15775 non-null  float64
 1   type          5661 non-null   object 
 2   address       15775 non-null  object 
 3   number        13577 non-null  float64
 4   postal_code   15775 non-null  int64  
 5   municipality  15775 non-null  object 
 6   province      15775 non-null  object 
 7   location      8962 non-null   object 
 8   public        8656 non-null   object 
 9   available     4739 non-null   object 
 10  hours         1148 non-null   object 
 11  full_address  15775 non-null  object 
 12  lat           15775 non-null  float64
 13  lon           15775 non-null  float64
dtypes: float64(4), int64(1), object(9)
memory usage: 1.7+ MB


In [7]:
hospital.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name_hospital  117 non-null    object 
 1   province       114 non-null    object 
 2   lat            116 non-null    float64
 3   lon            116 non-null    float64
dtypes: float64(2), object(2)
memory usage: 3.8+ KB


In [8]:
from pyproj import Geod # Calculate distance between two points on the earth

geod = Geod(ellps='WGS84')

def find_closest_distance(df1, df2):
    distances = []
    for index1, row1 in df1.iterrows():
        min_distance = float('inf')
        for index2, row2 in df2.iterrows():
            # calculate the distance between two points
            _, _, distance = geod.inv(row1['lon_itv'], row1['lat_itv'], row2['lon'],
                                      row2['lat'])
            # update the min distance
            if distance < min_distance:
                min_distance = distance
        # append the min distance to the list
        distances.append(min_distance)
    return distances



itv_aed['aed_distance'] = find_closest_distance(itv_aed, aed)
itv_aed.head()

,mission_id,service_name,postalcode_permanence,cityname_permanence,streetname_permanence,housenumber_permanence,latitude_permanence,longitude_permanence,permanence_short_name,permanence_long_name,...,t0_DayName,t7_Hour,t7_Day,t7_Month,t7_Year,t7_DayName,province,intervention_time_(t1confirmed),departure_time_(t1confirmed),aed_distance
0,20222490015,FB PDS BRUX [PASI CitÈ] SIAMU,1000.0,Brussel (Brussel),Vesaliusstraat,NaN,50.85097,4.364110,ABBRUX03A,AMB HELI 3,...,Tuesday,11.0,6.0,9.0,2022.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,61.129528
1,20222490041,HB UR BRUX CHU St Pierre,1000.0,Brussel (Brussel),Rue Haute,NaN,50.83433,4.345450,ABBRUX08A,AMB HSP 3,...,Tuesday,12.0,6.0,9.0,2022.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,71.815838
2,20222490041,HB UR BRUX CHU St Pierre,1000.0,Brussel (Brussel),Rue Haute,NaN,50.83433,4.345450,ABBRUX06A,AMB HSP 1,...,Tuesday,13.0,6.0,9.0,2022.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,71.815838
3,20222490042,FB PDS BRUX [PASI CitÈ] SIAMU,1000.0,Brussel (Brussel),Vesaliusstraat,NaN,50.85097,4.364110,ABBRUX11A,AMB CITE 2,...,Tuesday,NaN,NaN,NaN,NaN,NaN,Brussels Hoofdstedelijk Gewest,NaN,NaN,95.336554
4,20222490045,BB FORE Croix Rouge,1190.0,Vorst (Vorst),Rue Marconi,NaN,50.81598,4.341524,ABFORE01A,AMB FOREST 1,...,Tuesday,13.0,6.0,9.0,2022.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,165.716925


In [9]:
itv_aed['hospital_distance'] = find_closest_distance(itv_aed, hospital)
itv_aed.to_csv('/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/intervention_aed_related_distance.csv', index=False)

In [10]:
itv_ca = itv_aed[itv_aed['eventtype_trip'].isin([
    'P003 - Cardiac arrest'
    ])]

itv_ca.to_csv('/Users/lye/Downloads/MDA/Github-MDA2024/1_Data/CLEANED/intervention_ca_distance.csv', index=False)

In [11]:
itv_ca.drop_duplicates(subset=['lat_itv', 'lon_itv'], keep='first')

,mission_id,service_name,postalcode_permanence,cityname_permanence,streetname_permanence,housenumber_permanence,latitude_permanence,longitude_permanence,permanence_short_name,permanence_long_name,...,t7_Hour,t7_Day,t7_Month,t7_Year,t7_DayName,province,intervention_time_(t1confirmed),departure_time_(t1confirmed),aed_distance,hospital_distance
17,20222490200,FB PDS ANDE [PASI Anderlecht] SIAMU,1070.0,Anderlecht,Bergense Steenweg,NaN,50.832540,4.311990,ABANDE03A,AMB AND 3,...,18.0,6.0,9.0,2022.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,46.578200,3008.489539
25,20222500029,HB UR BRUX Iris Sud Etterbeek,1050.0,Elsene (Elsene),Rue Jean Paquot,NaN,50.825831,4.378717,UBIXEL01A,SMUR XL 1,...,1.0,7.0,9.0,2022.0,Wednesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,266.778697,221.735791
39,20222500165,HB UR BRUX St Jean,1000.0,Brussel (Brussel),Broekstraat,NaN,50.852300,4.359880,UBBRUX02A,SMUR STJEAN 2,...,11.0,7.0,9.0,2022.0,Wednesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,337.832103,1694.697981
82,20222510199,HB UR BRUX CHU ST-PIERRE (MED),1000.0,Bruxelles (Bruxelles),Rue Haute - Hoogstraat,NaN,50.835710,4.347930,UBBRUX01A,SMUR HSP 1,...,12.0,8.0,9.0,2022.0,Thursday,Brussels Hoofdstedelijk Gewest,NaN,NaN,171.188074,2529.736339
172,20222530014,HB UR BRUX CHU ST-PIERRE (MED),1000.0,Bruxelles (Bruxelles),Rue Haute - Hoogstraat,NaN,50.835710,4.347930,UBBRUX01A,SMUR HSP 1,...,NaN,NaN,NaN,NaN,NaN,Brussels Hoofdstedelijk Gewest,NaN,NaN,215.693375,3015.957876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105781,40231490790,HO UR AALS ASZ,NaN,NaN,NaN,NaN,50.936592,4.019092,UOAALS02A,MUG AALST 2,...,NaN,NaN,NaN,NaN,NaN,Provincie Oost-Vlaanderen,NaN,NaN,2604.666769,5465.013065
105785,40231500111,HO UR AALS ASZ,NaN,NaN,NaN,NaN,50.936592,4.019092,UOAALS02A,MUG AALST 2,...,NaN,NaN,NaN,NaN,NaN,Provincie Oost-Vlaanderen,20.0,1.0,306.251472,11796.494374
105799,40231500731,HO UR ZOTT AZ St Elisabeth,NaN,NaN,NaN,NaN,50.863941,3.817532,AOZOTT01A,ZW ZOTTEGEM,...,19.0,30.0,5.0,2023.0,Tuesday,Provincie Oost-Vlaanderen,11.0,1.0,2889.675943,6553.222587
105808,40231510072,HO UR GENT AZ Jan Palfijn,NaN,NaN,NaN,NaN,51.048772,3.701035,UOGENT04A,MUG GENT 4,...,NaN,NaN,NaN,NaN,NaN,Provincie Oost-Vlaanderen,14.0,2.0,346.496502,7932.285562


In [13]:
itv_aed.drop_duplicates(subset=['mission_id'], keep='first')

,mission_id,service_name,postalcode_permanence,cityname_permanence,streetname_permanence,housenumber_permanence,latitude_permanence,longitude_permanence,permanence_short_name,permanence_long_name,...,t7_Hour,t7_Day,t7_Month,t7_Year,t7_DayName,province,intervention_time_(t1confirmed),departure_time_(t1confirmed),aed_distance,hospital_distance
0,20222490015,FB PDS BRUX [PASI CitÈ] SIAMU,1000.0,Brussel (Brussel),Vesaliusstraat,NaN,50.850970,4.364110,ABBRUX03A,AMB HELI 3,...,11.0,6.0,9.0,2022.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,61.129528,465.046464
1,20222490041,HB UR BRUX CHU St Pierre,1000.0,Brussel (Brussel),Rue Haute,NaN,50.834330,4.345450,ABBRUX08A,AMB HSP 3,...,12.0,6.0,9.0,2022.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,71.815838,773.845722
3,20222490042,FB PDS BRUX [PASI CitÈ] SIAMU,1000.0,Brussel (Brussel),Vesaliusstraat,NaN,50.850970,4.364110,ABBRUX11A,AMB CITE 2,...,NaN,NaN,NaN,NaN,NaN,Brussels Hoofdstedelijk Gewest,NaN,NaN,95.336554,1529.142400
4,20222490045,BB FORE Croix Rouge,1190.0,Vorst (Vorst),Rue Marconi,NaN,50.815980,4.341524,ABFORE01A,AMB FOREST 1,...,13.0,6.0,9.0,2022.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,165.716925,1306.596211
5,20222490053,HB UR BRUX Europe Michel,1040.0,Etterbeek (Etterbeek),Charles Degrouxstraat,NaN,50.842260,4.399250,ABETTE01A,AMB ST MICHEL,...,13.0,6.0,9.0,2022.0,Tuesday,Brussels Hoofdstedelijk Gewest,NaN,NaN,149.195110,566.010100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105834,40231510870,HO UR ZOTT AZ St Elisabeth,NaN,NaN,NaN,NaN,50.863941,3.817532,UOZOTT01A,MUG ZOTTEGEM,...,NaN,NaN,NaN,NaN,NaN,Provincie Oost-Vlaanderen,9.0,5.0,2288.795551,4505.682461
105836,40231510884,MO HVP BEVE Waasland,NaN,NaN,NaN,NaN,51.210718,4.259800,POBEVE01A,PIT BEVEREN 1,...,NaN,NaN,NaN,NaN,NaN,Provincie Oost-Vlaanderen,3.0,0.0,198.770237,9307.349422
105837,40231510891,BO LOCH 112 Lochristi,NaN,NaN,NaN,NaN,51.097770,3.830819,AOLOCH01A,ZW LOCHRISTI 1,...,NaN,NaN,NaN,NaN,NaN,Provincie Oost-Vlaanderen,5.0,2.0,651.924810,8087.054410
105838,40231510894,HO UR GENT AZ Maria Middelares,NaN,NaN,NaN,NaN,51.027962,3.700208,POMERE01B,PIT MERELBEKE ZH,...,NaN,NaN,NaN,NaN,NaN,Provincie Oost-Vlaanderen,14.0,5.0,759.531866,7018.075854
